In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.datasets import cifar100

In [3]:
batch_size = 64
buffersize = 50000
steps_per_epoch = int(np.ceil(50000) / float(batch_size))
epochs = 10
num_classes = 100

(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar100.load_data(label_mode="fine")

X_train = X_train_orig.astype(np.float32)/255
X_test = X_test_orig.astype(np.float32)/255
Y_train = tf.one_hot(Y_train_orig, num_classes)
Y_test = tf.one_hot(Y_train_orig, num_classes)

print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

X_train: (50000, 32, 32, 3)
Y_train: (50000, 1, 100)
X_test:  (10000, 32, 32, 3)
Y_test:  (50000, 1, 100)


In [8]:
def Conv_Model(input_shape):
    Conv_Model = models.Sequential()
    Conv_Model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape = input_shape))
    Conv_Model.add(layers.LeakyReLU())
    Conv_Model.add(layers.Dropout(0.3))

    Conv_Model.add(layers.MaxPooling2D((2,2)))

    Conv_Model.add(layers.Conv2D(128, (3,3), activation='relu'))
    Conv_Model.add(layers.LeakyReLU())
    Conv_Model.add(layers.Dropout(0.3))

    Conv_Model.add(layers.MaxPooling2D((2,2)))

    Conv_Model.add(layers.Conv2D(256, (3,3), activation='relu'))
    Conv_Model.add(layers.LeakyReLU())
    Conv_Model.add(layers.Dropout(0.3))

    Conv_Model.add(layers.MaxPooling2D((2,2)))

    Conv_Model.add(layers.Flatten())
    Conv_Model.add(layers.Dense(256, activation='relu'))
    Conv_Model.add(layers.Dense(128, activation='relu'))
    Conv_Model.add(layers.Dense(100, activation='softmax'))

    return Conv_Model

In [ ]:
Conv_Model = Conv_Model(X_train.shape)
loss = keras.losses.sparse_categorical_crossentropy()
Conv_Model.compile(optimizer='adam', loss= loss, metrics=['accuracy'])